In [1]:
import os 
from openai import OpenAI

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
endpoint = "https://models.github.ai/inference"
model_name = "openai/gpt-4o"

client = OpenAI(
    base_url=endpoint,
    api_key=os.environ['GITHUB_API_TOKEN'],
    )

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of England?",
        }
    ],
    temperature=0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name

)



In [5]:
from typing import List
from pydantic import BaseModel, Field

In [6]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [7]:
foo = User(name="John Doe", age=30, email="jondon@gmail.com")

In [8]:
foo.name

'John Doe'

In [9]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [10]:
foo.age

'bar'

In [11]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [12]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [13]:
foo_p.name

'Jane'

In [14]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing

In [15]:
class Class(BaseModel):
    students: List[pUser]

In [16]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [17]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

### Pydantic to OpenAI function definition

In [19]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport."""
    airport_code: str = Field(description="The airport code to get the weather for.")

In [23]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [24]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [25]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport.',
 'parameters': {'properties': {'airport_code': {'description': 'The airport code to get the weather for.',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [26]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [28]:
convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [31]:
from langchain_openai.chat_models import ChatOpenAI

In [34]:
model = ChatOpenAI(model="gpt-4o", temperature=0)

In [35]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 70, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZwUvb90ahxDvo8gkI8OUIw1cxnKx', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--5fb2c208-647f-4fb6-b036-a476572af9cd-0', usage_metadata={'input_tokens': 70, 'output_tokens': 18, 'total_tokens': 88, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [36]:
model_with_function = model.bind(functions=[weather_function])

In [37]:
model_with_function.invoke("what is the weather in SF today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 70, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZwVooIBEitC6B4tJoTBt19Yyqsjw', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--e709eae5-1893-4024-96ed-2ae56144131e-0', usage_metadata={'input_tokens': 70, 'output_tokens': 18, 'total_tokens': 88, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Forcing it to use a function

In [38]:
model_with_force_function = model.bind(functions=[weather_function], function_call={"name": "WeatherSearch"})

In [39]:
model_with_force_function.invoke("what is the weather in SF today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 80, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZwZqO0VnrHvYX1akjauyCZLeeGxz', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--9a8588b5-5302-4e58-8a2f-46cbd2206ea7-0', usage_metadata={'input_tokens': 80, 'output_tokens': 8, 'total_tokens': 88, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [40]:
model_with_force_function.invoke("Hello!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 74, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZwaSRkOrVIJoR2sLqLwalGlcnXlt', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--e3633760-67d3-4053-bbd2-4da040b1c591-0', usage_metadata={'input_tokens': 74, 'output_tokens': 8, 'total_tokens': 82, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Using in a chain

In [41]:
from langchain.prompts import ChatPromptTemplate

In [42]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}"),
])

In [43]:
chain = prompt | model_with_function

In [44]:
chain.invoke({"input": "what is the weather in SF today?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 76, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZwcevMc7oPAmJxJmykTZedEg9lfH', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--928f7538-65fc-4c71-bc04-bdf6399413d1-0', usage_metadata={'input_tokens': 76, 'output_tokens': 18, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Using multiple functions

In [45]:
class ArtistSearch(BaseModel):
    """Call this with an artist name to get the top 5 songs by that artist."""
    artist_name: str = Field(description="The name of the artist to search for.")
    n : int = Field(default=5, description="The number of songs to return.")

In [46]:
functions =[
    convert_pydantic_to_openai_function(ArtistSearch),
    convert_pydantic_to_openai_function(WeatherSearch)
]

In [47]:
model_with_functions = model.bind(functions=functions)

In [48]:
model_with_functions.invoke("what is the weather in SF today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 129, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZweKjcIrnegj6mtaUArIeiv0JWl4', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--1568ebe7-c5ca-4260-9714-5e7b34c233ea-0', usage_metadata={'input_tokens': 129, 'output_tokens': 18, 'total_tokens': 147, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [50]:
model_with_functions.invoke("what are three songs by sergio Mendes?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Sergio Mendes","n":3}', 'name': 'ArtistSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 130, 'total_tokens': 154, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZwemEDtMMqPLx0QW3n0OIAEKmIdX', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--506fc0ce-985f-49b1-9a47-3a24d5294fab-0', usage_metadata={'input_tokens': 130, 'output_tokens': 24, 'total_tokens': 154, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [51]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 123, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZweq31VwSLY91zktTaDgSwCs6NDW', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--89572fa0-668a-4a9b-85ab-be346335cbda-0', usage_metadata={'input_tokens': 123, 'output_tokens': 12, 'total_tokens': 135, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})